In [0]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import pandas as pd
torch.manual_seed(1)
import os,csv,math, sys,joblib,copy,pickle
import matplotlib.pyplot as plt
import numpy as np
import sklearn.model_selection, sklearn.preprocessing

In [0]:
seed = 1
np.random.seed(seed)
torch.manual_seed(seed)

In [0]:
dataset = pd.read_csv('Iris_Dataset.csv')

In [0]:
dataset.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
dataset = pd.get_dummies(dataset, columns=['Species']) # One Hot Encoding
values = list(dataset.columns.values)

In [0]:
dataset.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species_Iris-setosa,Species_Iris-versicolor,Species_Iris-virginica
0,5.1,3.5,1.4,0.2,1,0,0
1,4.9,3.0,1.4,0.2,1,0,0
2,4.7,3.2,1.3,0.2,1,0,0
3,4.6,3.1,1.5,0.2,1,0,0
4,5.0,3.6,1.4,0.2,1,0,0


values

In [0]:
y = dataset[values[-3:]]
y = np.array(y, dtype='float32')
#y

In [0]:
X = dataset[values[:-3]]
print X.head()
X = np.array(X, dtype='float32')

   SepalLengthCm  SepalWidthCm  PetalLengthCm  PetalWidthCm
0            5.1           3.5            1.4           0.2
1            4.9           3.0            1.4           0.2
2            4.7           3.2            1.3           0.2
3            4.6           3.1            1.5           0.2
4            5.0           3.6            1.4           0.2


In [0]:
mean_vector=np.mean(X,axis=0)
#means.shape#4
cov=np.cov(X.T)
cov.shape

(4, 4)

In [0]:
indices = np.random.choice(len(X), len(X), replace=False)
X_values = X[indices]

scaler = joblib.load('scaler.pkl') 
X_values = scaler.transform(X_values)
y_values = y[indices]

In [0]:
class neural_network(nn.Module):
    def __init__(self, num_input, num_hidden,num_output):
        super(neural_network, self).__init__()
        self.hidden = nn.Linear(num_input,num_hidden).cuda()
        self.out = nn.Linear(num_hidden,num_output).cuda()

    def forward(self, x):
        x = F.relu(self.hidden(x))
        return self.out(x)

In [0]:
model = torch.load("iris_model_2").cpu()

In [0]:
n_classes=3
num_c=4#no. of features
num_alpha=100

for output_index in range(0,n_classes):
    for t in range(0,num_c):
        expected_value = 0.0
        counter = 1
        expectation_do_x = []
        baseline_expectation_do_x = []
        inp=copy.deepcopy(mean_vector)
        
        for x in np.linspace(0, 1, num_alpha):
            inp[t] = x
            input_torchvar = autograd.Variable(torch.FloatTensor(inp), requires_grad=True)
            
            output=F.softmax(model(input_torchvar), dim=-1) 
            o1=output.data.cpu()
            val=o1.numpy()[output_index]                                              
            grad_mask_gradient = torch.zeros(n_classes)
            grad_mask_gradient[output_index] = 1.0
            #calculating the hessian
            first_grads = torch.autograd.grad(output.cpu(), input_torchvar.cpu(), grad_outputs=grad_mask_gradient, retain_graph=True, create_graph=True)

            for dimension in range(num_c):#Hessian*Covariance
                if dimension == t:
                        continue
                temp_cov = copy.deepcopy(cov)
                temp_cov[dimension][t] = 0.0
                grad_mask_hessian = torch.zeros(num_c)#SHOULD IT BE num_classes
                grad_mask_hessian[dimension] = 1.0

                #calculating the hessian
                hessian = torch.autograd.grad(first_grads, input_torchvar, grad_outputs=grad_mask_hessian, retain_graph=True, create_graph=False)

                val += np.sum(0.5*hessian[0].data.numpy()*temp_cov[dimension])  #Has interventional expectations of x on y for an alpha
            expectation_do_x.append(val)

            expected_value += (1.0/counter)*(val - expected_value)
            counter += 1
        baseline_expectation_do_x.append(expected_value)
        if not os.path.exists('ACE'):
          os.mkdir('ACE')
        with open('ACE/expectation_do_x_for_cause_'+str(t) + "_and_effect_" + str(output_index), 'wb') as fp:
            pickle.dump(expectation_do_x, fp)
        with open('ACE/baseline_expectation_do_x_for_cause_'+str(t) + "_and_effect_" + str(output_index), 'wb') as fp:
            pickle.dump(baseline_expectation_do_x, fp)